# Ejemplo: Programación Evolutiva (PE)

## Función de Beale


La función de Beale es una función de dos variables y está definida como sigue:

\begin{equation}
  \min f(x_1, x_2) = (1.5 - x_1 + x_1 x_2)^2 + (2.25 - x_1 + x_1 x_2^2)^2 + (2.625 - x_1 + x_1 x_2^3)^2
\end{equation}

donde $-4.5 \leq x_1, x_2 \leq 4.5$. El mínimo global está en $x^* = (3, 0.5)$ y $f(x^*)=0$.

## Principales componentes de PE

Librería de Python que vamos a utilizar.

In [1]:
import numpy as np

Datos de entrada.

In [2]:
n = 2 #Número de variables de decisión

### Representación de un individuo

Un arreglo de valores flotantes de tamaño 4. Las primeras dos componentes corresponden a las dos variables del problema. Las siguentes dos componentes corresponden a los tamaños de paso para la mutación de cada una de las variables del problema.

In [3]:
i = np.array([-1.5, 2.8, 0.3, 0.15])

### Aptitud

La aptitud de un individuo $i$ la definimos como $F_a(i) = -f(x^{(i)}_1, x^{(i)}_2)$, donde $f$ es la función de Beale.

In [4]:
def f(x):
    x1, x2 = x[0], x[1]
    term1 = (1.5 - x1 + x1*x2)**2
    term2 = (2.25 - x1 + x1*(x2**2))**2
    term3 = (2.625 - x1 + x1*(x2**3))**2

    return term1 + term2 + term3

def fitness(x):
    return -f(x)

In [5]:
print ("x = (%s, %s)\t-f(x) = %s\t"%(i[0], i[1], fitness(i[:2])))

x = (-1.5, 2.8)	-f(x) = -895.2129089999994	


### Población inicial

Por cada individuo requerido, se utiliza una distribución uniforme para generar números aleatorios. Primero se generan dos números en el intervalo $[-4.5, 4.5]$ para crear las variables de decisión. Posteriormente, se generan dos números en el intervalo $(0,1)$ para crear los tamaños de paso.

In [6]:
def getInitialPopulation(mu, n):
    parents = []
    for i in range(mu):
        #Generamos un individuo
        p = np.concatenate((np.random.uniform(-4.5, 4.5, n), np.random.uniform(0, 1, n)))
        #Calculamos la aptitud del individuo
        p = [p, fitness(p[:n])] 
        #Agregamos al individuo a la población "parents"
        parents.append(p)
    
    #Regresamos la población generada
    return parents

In [7]:
mu = 100 #Tamaño de la población
population = getInitialPopulation(mu, n)
print("Tamaño de la población:", len(population))
print("Primer individuo generado: ", population[0])

Tamaño de la población: 100
Primer individuo generado:  [array([-3.57073909,  4.48033254,  0.04512549,  0.43092328]), -103643.736219691]


### Mutación y autoadaptación

Para mutar al individuo, se necesita primero mutar los valores de los tamaño de paso  de la siguiente forma:

  \begin{eqnarray*}
    \sigma'_1 = \sigma_1 \cdot (1 + \alpha \cdot N(0,1)) \\
    \sigma'_2 = \sigma_2 \cdot (1 + \alpha \cdot N(0,1))
  \end{eqnarray*}

verificamos que los valores de $\sigma'_1$ y $\sigma'_2$ no sean menores que $\varepsilon_0$, en caso de que sí, se dejarán en $\varepsilon_0$. Posteriormente, utilizamos los nuevos $\sigma$ para mutar las variables de decisión como sigue:

\begin{eqnarray*}
    x'_1 = x_1 + \sigma'_1 \cdot N(0,1) \\
    x'_2 = x_2 + \sigma'_2 \cdot N(0,1) 
\end{eqnarray*}

Finalmente, construimos el individuo hijo:

\begin{equation*}
    i' = [x'_1, x'_2, \sigma'_1, \sigma'_2]
  \end{equation*}

Cuando se mutan las variables de decisión, se debe revisar que estén dentro del rango que indica el problema. En caso de que no, se le asignará el valor del límite que rebasó.

In [10]:
def mutation(i, n, alpha, epsilon):
    #Mutamos los últimos dos componentes de nuestro arreglo 
    #(tamaños de paso) 
    mutation_sigma = i[n:]*(1+(alpha*np.random.normal(0, 1, n)))
    #Verificamos que los nuevos valores no sean menores a épsilon
    mutation_sigma[mutation_sigma < epsilon] = epsilon

    #Mutamos las variables de decisión a partir de las mutaciones
    mutation_x = i[:n] + (mutation_sigma*np.random.normal(0, 1, n))

    #Revisamos que estén dentro de los límites
    mutation_x[mutation_x < -4.5] = -4.5
    mutation_x[mutation_x >  4.5] =  4.5

    #Creamos el nuevo Individuo y lo devolvemos
    return [np.concatenate((mutation_x, mutation_sigma)), fitness(mutation_x)]

In [11]:
#Parámetros para la mutación
alpha = 0.2 
epsilon = 0.01

child = mutation(population[0][0], n, alpha, epsilon)
print(child)

[array([-3.46492722,  3.83356143,  0.06230124,  0.43211314]), -37879.09011226479]


## Algoritmo completo de PE simple

A partir de $100$ padres, vamos a generar $100$ hijos. Unimos ambas poblaciones y seleccionamos los $100$ mejores individuos de acuerdo a su aptitud.

In [12]:
def EvolutionaryProgramming(n, mu, G, alpha, epsilon):
    parents = getInitialPopulation(mu, n)

    for t in range(num_gen):
        new_gen = parents.copy()

        for parent in parents:
            #Creamos un hijo
            child = mutation(parent[0], n, alpha, epsilon)
            #Agregamos al hijo a la nueva generación
            new_gen.append(child)
        
        #Ordenamos del peor individuo al mejor individuo 
        #(del menor valor de aptitud al mayor)
        new_gen = sorted(new_gen, key=lambda individual: individual[-1])
        #Nos quedamos con los mu mejores individuos
        new_gen = new_gen[mu:]
        
        parents = new_gen.copy()

    #devolvemos el mejor
    return parents[-1][0], -parents[-1][1]

In [16]:
n = 2 
mu = 100 
G = 200 
alpha = 0.2 
epsilon = .01 

In [14]:
sol, fx = EvolutionaryProgramming(n, mu, G, alpha, epsilon)
print(f"x: {sol[:n]}\nsigma: {sol[n:]}\nf(x): {fx}")

x: [2.99992434 0.50001577]
sigma: [0.01       0.01466159]
f(x): 2.8418807729833617e-08


Realizamos $m$ ejecuciones de nuestro algoritmo y obtenemos la corrida que encontró la mejor solución, la corrida que encontró la peor solución, la corrida que se encuentra en la mediana de las $m$ corridas, la media y la desviación estándar de las $m$ corridas con respecto al valor de la función objetivo.

In [15]:
m = 21
sol =[]
for m in range(m):
    x, fx = EvolutionaryProgramming(n, mu, num_gen, alpha, epsilon)
    sol.append([x, fx])
    
sol = sorted(sol, key=lambda individuo: individuo[-1])
print("*************** Mejor solución ***************")
print(f"x: {sol[0][0][:n]}\nsigma: {sol[0][0][n:]}\nf(x): {sol[0][-1]}")

print("\n*************** Peor solución ****************")
print(f"x: {sol[-1][0][:n]}\nsigma: {sol[-1][0][n:]}\nf(x): {sol[-1][-1]}")

print("\n****************** Mediana *******************")
print(f"x: {sol[m//2][0][:n]}\nsigma: {sol[m//2][0][n:]}\nf(x): {sol[m//2][-1]}")

f_sol = [x[-1] for x in sol]
print("\nMedia: ", np.mean(f_sol))
print("\nDesviación estándar: ", np.std(f_sol))

*************** Mejor solución ***************
x: [2.99971315 0.49991913]
sigma: [0.02772399 0.01081308]
f(x): 1.5359162146571287e-08

*************** Peor solución ****************
x: [2.999368   0.49990788]
sigma: [0.03042325 0.02163403]
f(x): 1.6027470532192182e-07

****************** Mediana *******************
x: [2.99950765 0.49988047]
sigma: [0.01059131 0.01625244]
f(x): 3.897568350940374e-08

Media:  5.586687828982277e-08

Desviación estándar:  3.49564794851159e-08


Recordemos que esta es la versión más simple de PE, se pueden hacer cambios en el diseño para mejorar la eficacia del algoritmo.